In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf

/Users/mani/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from image_converter import get_x_shape

### create session

In [3]:
sess = tf.InteractiveSession()

### create graph placement

In [4]:
width = 224 # width of the image in pixels 
height = 224 # height of the image in pixels
flat = width * height # number of pixels in one image 
class_output = 14951 # number of possible classifications for the problem

In [5]:
x_shape = get_x_shape('b399f09dee9c3c67.jpeg')
print(x_shape)

[None, 224, 224, 3]


In [6]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [8]:
x_image = tf.reshape(x, [-1,224,224,3])  
x_image

<tf.Tensor 'Reshape_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [9]:
y_.shape

TensorShape([Dimension(None), Dimension(14951)])

### 1)Layer 1: conv 3x3 64 (RELU)

- Input: 224x224x3
- Type: Conv
- size: 3x3
- channel: 64
- strides: 1
- Output: 224x224x64

In [11]:
W_conv1 = tf.Variable(tf.truncated_normal([3, 3, 3, 64], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))

In [12]:
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1

In [13]:
h_conv1 = tf.nn.relu(convolve1)

### 2)Layer 1: max pooling 2x2 2

- Input: 224x224x64
- pool: 2x2
- output: 112x112x64

In [14]:
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
conv1

<tf.Tensor 'MaxPool:0' shape=(?, 112, 112, 64) dtype=float32>

### 3)Layer 2: conv 3x3 128 (RELU)

- Input: 112x112x64
- Type: Conv
- size: 3x3
- channel: 128
- strides: 1
- Output: 112x112x128

In [15]:
W_conv2 = tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[128]))

In [16]:
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2

In [17]:
h_conv2 = tf.nn.relu(convolve2)

### 4)Layer 2: max pooling 2x2 2

- Input: 112x112x128
- pool: 2x2
- output: 56x56x128

In [18]:
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
conv2

<tf.Tensor 'MaxPool_1:0' shape=(?, 56, 56, 128) dtype=float32>

### 5)Layer 3: conv 3x3 256 (RELU)

- Input: 56x56x128
- Type: Conv
- size: 3x3
- channel: 256
- strides: 1
- Output: 56x56x256

In [23]:
W_conv3 = tf.Variable(tf.truncated_normal([3, 3, 128, 256], stddev=0.1))
b_conv3 = tf.Variable(tf.constant(0.1, shape=[256]))

In [24]:
convolve3= tf.nn.conv2d(conv2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')+ b_conv3

In [21]:
h_conv3 = tf.nn.relu(convolve3)

### 6)Layer 3: max pooling 2x2 2

- Input: 56x56x256
- pool: 2x2
- output: 28x28x256

In [25]:
conv3 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
conv3

<tf.Tensor 'MaxPool_3:0' shape=(?, 28, 28, 256) dtype=float32>

### 7)Layer 4: conv 3x3 512 (RELU)

- Input: 28x28x256
- Type: Conv
- size: 3x3
- channel: 512
- strides: 1
- Output: 28x28x512

In [26]:
W_conv4 = tf.Variable(tf.truncated_normal([3, 3, 256, 512], stddev=0.1))
b_conv4 = tf.Variable(tf.constant(0.1, shape=[512]))

In [27]:
convolve4= tf.nn.conv2d(conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME')+ b_conv4

In [28]:
h_conv4 = tf.nn.relu(convolve4)

### 8)Layer 4: max pooling 2x2 2

- Input: 28x28x512
- pool: 2x2
- output: 14x14x512

In [29]:
conv4 = tf.nn.max_pool(h_conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
conv4

<tf.Tensor 'MaxPool_4:0' shape=(?, 14, 14, 512) dtype=float32>

### 9)Layer 5: conv 3x3 512 (RELU)

- Input: 14x14x512
- Type: Conv
- size: 3x3
- channel: 512
- strides: 1
- Output: 14x14x512

In [30]:
W_conv5 = tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev=0.1))
b_conv5 = tf.Variable(tf.constant(0.1, shape=[512]))

In [31]:
convolve5= tf.nn.conv2d(conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME')+ b_conv5

In [32]:
h_conv5 = tf.nn.relu(convolve5)

### 10)Layer 5: max pooling 2x2 2

- Input: 14x14x512
- pool: 2x2
- output: 7x7x512

In [33]:
conv5 = tf.nn.max_pool(h_conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
conv5

<tf.Tensor 'MaxPool_5:0' shape=(?, 7, 7, 512) dtype=float32>

### 11)Layer 6: Fully Connected Layer 4096 (ReLU)

reshape:[1, 7x7x512 = 25088]
output: [1, 4096]

In [34]:
layer5_matrix = tf.reshape(conv5, [-1, 25088])

In [35]:
W_fc1 = tf.Variable(tf.truncated_normal([25088, 4096], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[4096])) 

In [36]:
fcl=tf.matmul(layer5_matrix, W_fc1) + b_fc1

In [37]:
h_fc1 = tf.nn.relu(fcl)
h_fc1

<tf.Tensor 'Relu_5:0' shape=(?, 4096) dtype=float32>

### 12)Layer 7: Fully Connected Layer - Dropout Layer (reducing overfitting)

In [38]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 4096) dtype=float32>

###  13)Layer 8: Readout Layer (Softmax Layer)

In [39]:
W_fc2 = tf.Variable(tf.truncated_normal([4096, 14951], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[14951])) 

In [40]:
fc=tf.matmul(layer_drop, W_fc2) + b_fc2

In [41]:
y_CNN= tf.nn.softmax(fc)
y_CNN

<tf.Tensor 'Softmax:0' shape=(?, 14951) dtype=float32>

# Define functions and train the model